In [11]:
#Setup Paths

In [12]:
import sys, os, importlib.util

# Go 2 levels up (to project root)
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
backend_path = os.path.join(project_root, "backend")

print("Project root:", project_root)
print("Backend path:", backend_path)

Project root: c:\Users\USER\OneDrive\Desktop\Infinity AI\ApparelWeb\InfinityStyleVerse
Backend path: c:\Users\USER\OneDrive\Desktop\Infinity AI\ApparelWeb\InfinityStyleVerse\backend


In [13]:
#Import Functions Without Flask

In [14]:
# Load experiment_data.py
exp_data_path = os.path.join(backend_path, "app", "utils", "experiment_data.py")
spec = importlib.util.spec_from_file_location("experiment_data", exp_data_path)
exp_data = importlib.util.module_from_spec(spec)
spec.loader.exec_module(exp_data)

# Load experimentation.py
experimentation_path = os.path.join(backend_path, "app", "services", "experimentation.py")
spec2 = importlib.util.spec_from_file_location("experimentation", experimentation_path)
experimentation = importlib.util.module_from_spec(spec2)
spec2.loader.exec_module(experimentation)

# Assign functions for convenience
fetch_experiment_data = exp_data.fetch_experiment_data
calculate_cuped_adjusted_metric = experimentation.calculate_cuped_adjusted_metric

print("Functions imported successfully!")


Functions imported successfully!


In [15]:
#Simulate A/A Test

In [16]:
import pandas as pd
import numpy as np

# Generate synthetic data
df = fetch_experiment_data("exp_aa", num_samples=2000)

# Force no true effect
df["outcome"] = 0.5 + 0.2 * df["pre_metric"] + np.random.normal(0, 0.3, size=len(df))

# Split into groups
control = df[df["treatment"] == 0]
treatment = df[df["treatment"] == 1]

df.head()

,experiment_id,treatment,pre_metric,outcome
0,exp_aa,0,-0.877983,0.272615
1,exp_aa,1,-0.826880,0.526227
2,exp_aa,1,-0.226479,0.435632
3,exp_aa,1,0.367366,0.138195
4,exp_aa,0,0.913585,0.705390


In [17]:
#Run CUPED

In [18]:
results = calculate_cuped_adjusted_metric(
    control_metric=control["outcome"].values,
    control_covariate=control["pre_metric"].values,
    treatment_metric=treatment["outcome"].values,
    treatment_covariate=treatment["pre_metric"].values
)

print("CUPED Results:", results)


CUPED Results: {'theta': np.float64(0.1941080430411694), 'control_mean_raw': np.float64(0.49020835722199707), 'treatment_mean_raw': np.float64(0.4959582171703441), 'control_mean_cuped': np.float64(0.49269687226501657), 'treatment_mean_cuped': np.float64(0.49353357319456886), 'uplift_raw': np.float64(0.005749859948347036), 'uplift_cuped': np.float64(0.000836700929552292), 'var_reduction': np.float64(0.2802631392600908)}
